In [1]:
import numpy as np
import xarray as xr

##--##--##--##--  读取 15 天滑动气温 10%, 90% 阈值 (JJA)   --##--##--##--##
ds00 = xr.open_dataset("/home/ys17-23/cai_fy/data4_heatwaves/NCC2_1980_2010/1_threshold/era5_Tmax_th90_JJA_1981_2010.nc")
t_th90 = ds00.air_th90.loc[:, 90:0,:][:,::-1,:]


##--##--##--##--  读取 ERA5 2米气温数据  --##--##--##--##
ds00 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m/t2m.2021-05.daily.nc")
t0 = ds00.t2m.loc[:,90:0,:][:,::-1,:]; ds00.close()
lat = t0.latitude; lon = t0.longitude
print(np.shape(t0))
t = np.zeros((44, 92, np.shape(t0)[1],np.shape(t0)[2]))


for iyear in range(44):
  print('iyear = ',iyear+1979)
  ds2 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m_max/t2m."+str(iyear+1979)+"-06.daily.nc")
  t[iyear,0:30,:,:] = ds2.t2m.loc[:,90:0,:][:,::-1,:]
  ds3 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m_max/t2m."+str(iyear+1979)+"-07.daily.nc")
  t[iyear,30:61,:,:] = ds3.t2m.loc[:,90:0,:][:,::-1,:]
  ds4 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m_max/t2m."+str(iyear+1979)+"-08.daily.nc")
  t[iyear,61:92,:,:] = ds4.t2m.loc[:,90:0,:][:,::-1,:]
  ds2.close(); ds3.close(); ds4.close()

/home/soft/anaconda3/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.21.0 or higher is recommended. You are running version 2.17.0
  warnings.warn(


(31, 91, 360)
iyear =  1979
iyear =  1980
iyear =  1981
iyear =  1982
iyear =  1983
iyear =  1984
iyear =  1985
iyear =  1986
iyear =  1987
iyear =  1988
iyear =  1989
iyear =  1990
iyear =  1991
iyear =  1992
iyear =  1993
iyear =  1994
iyear =  1995
iyear =  1996
iyear =  1997
iyear =  1998
iyear =  1999
iyear =  2000
iyear =  2001
iyear =  2002
iyear =  2003
iyear =  2004
iyear =  2005
iyear =  2006
iyear =  2007
iyear =  2008
iyear =  2009
iyear =  2010
iyear =  2011
iyear =  2012
iyear =  2013
iyear =  2014
iyear =  2015
iyear =  2016
iyear =  2017
iyear =  2018
iyear =  2019
iyear =  2020
iyear =  2021
iyear =  2022


In [2]:
##--##--##--##--  对于某一时刻，气温是否超过90%阈值, 或低于10%阈值  --##--##--##--##
t_True = np.zeros((44, 92, np.shape(t0)[1],np.shape(t0)[2]))
Intensity_True = np.zeros((44, 92, np.shape(t0)[1],np.shape(t0)[2]))
t_True[:,:,:,:] = np.array(t) >= np.array(t_th90)
Intensity_True[:,:,:,:] = np.array(t) - np.array(t_th90)
del t; print("Heat days",np.sum(t_True), np.size(t_True))


##--##--##--##--  热浪事件：连续三天超过90%阈值； 偏冷事件：或低于10%阈值  --##--##--##--##
t_True2 = t_True.copy()
t_True2[:,2:-2,:,:][t_True[:,:-4,:,:]+t_True[:,1:-3,:,:]+t_True[:,3:-1,:,:]+t_True[:,4:,:,:]<=1] = 0.0
t_True2[:,2:-2,:,:][t_True[:,1:-3,:,:]+t_True[:,3:-1,:,:]==0] = 0.0
t_True2[:,2:-2,:,:][(t_True[:,1:-3,:,:]+t_True[:,3:-1,:,:]==1)&(t_True[:,3:-1,:,:]+t_True[:,4:,:,:]<=1)&(t_True[:,:-4,:,:]+t_True[:,1:-3,:,:]<=1)] = 0.0

t_True2[:,0,:,:][t_True[:,1,:,:]+t_True[:,2,:,:]<=1] = 0.0
t_True2[:,1,:,:][t_True[:,0,:,:]+t_True[:,2,:,:]+t_True[:,3,:,:]<=1] = 0.0
t_True2[:,1,:,:][(t_True[:,0,:,:]==1)&(t_True[:,2,:,:]==0)&(t_True[:,3,:,:]==1)] = 0.0
t_True2[:,-1,:,:][t_True[:,-2,:,:]+t_True[:,-3,:,:]<=1] = 0.0
t_True2[:,-2,:,:][t_True[:,-1,:,:]+t_True[:,-3,:,:]+t_True[:,-4,:,:]<=1] = 0.0
t_True2[:,-2,:,:][(t_True[:,-1,:,:]==1)&(t_True[:,-3,:,:]==0)&(t_True[:,-4,:,:]==1)] = 0.0
del t_True



##--##--##--##--  热浪事件强度：超过90%阈值程度； 偏冷事件强度：或低于10%阈值程度  --##--##--##--##
Intensity_True = Intensity_True * np.array(t_True2)
print("Heat Intensity ",np.sum(Intensity_True), np.max(Intensity_True), np.min(Intensity_True))

Heat days 15968516.0 132612480
Heat Intensity  10112629.718646264 19.810333251953125 -0.0


In [3]:
##--##--##--##--  每个格点，每个夏天的热浪天数 (year * lat * lon)  --##--##--##--##
heatwave_day = np.nansum(np.array(t_True2), axis=1)
print(np.shape(heatwave_day), np.max(heatwave_day), np.min(heatwave_day))

heatwave_intensity = np.nansum(np.array(Intensity_True), axis=1)
print(np.shape(heatwave_intensity), np.max(heatwave_intensity), np.min(heatwave_intensity))


##--##--##--##--  每年热浪天数, 存入nc文件  --##--##--##--##
heatwave_day_array = xr.DataArray(data=heatwave_day.data, dims=['year', 'lat', 'lon'],
                                coords={'year':np.linspace(1979,2022,44), 'lat':lat.data, 'lon':lon.data})
heatwave_intensity_array = xr.DataArray(data=heatwave_intensity.data, dims=['year', 'lat', 'lon'],
                                coords={'year':np.linspace(1979,2022,44), 'lat':lat.data, 'lon':lon.data})

ds0 = xr.Dataset(data_vars=dict(heatwave_day=heatwave_day_array, heatwave_intensity=heatwave_intensity_array))
ds0.to_netcdf("/home/ys17-23/cai_fy/data4_heatwaves/NCC2_1980_2010/1_threshold/era5_Tmax_heatwave_intensity_yearly.nc")
ds0.close()

(44, 91, 360) 92.0 0.0
(44, 91, 360) 346.83034362792955 0.0
